In [1]:
import fire
import os
import sys
import time
import json
import re
import random
import logging

import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from vllm import LLM, SamplingParams

from inference_utils import get_user_prompt, post_processing_answer
from embeddings_space import *

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-06 14:49:53,710	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-06 14:49:53,780	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def main(
    model_path,
    # model_name,
    # peft_model: str = None,
    quantization: bool = False,
    load_in: str = "4bit",
    max_new_tokens=100,  # The maximum numbers of tokens to generate
    test_file: str = "datasets/math_test.json",
    seed: int = 42,  # seed value for reproducibility
    do_sample: bool = True,  # Whether or not to use sampling ; use greedy decoding otherwise.
    min_length: int = None,  # The minimum length of the sequence to be generated, input prompt + min_new_tokens
    use_cache: bool = True,  # [optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
    top_p: float = 1.0,  # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    temperature: float = 1.0,  # [optional] The value used to modulate the next token probabilities.
    top_k: int = 50,  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
    repetition_penalty: float = 1.0,  # The parameter for repetition penalty. 1.0 means no penalty.
    length_penalty: int = 1,  # [optional] Exponential penalty to the length that is used with beam-based generation.
    enable_azure_content_safety: bool = False,  # Enable safety check with Azure content safety api
    enable_sensitive_topics: bool = False,  # Enable check for sensitive topics using AuditNLG APIs
    enable_salesforce_content_safety: bool = True,  # Enable safety check with Salesforce safety flan t5
    max_padding_length: int = None,  # the max padding length to be used with tokenizer padding the prompts.
    use_fast_kernels: bool = False,  # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels
    log_filename: str = "log.txt",
    output_filepath: str = "jupyter_submission.csv",
    time_output_filepath: str = "time_submission.csv",
    **kwargs,
):
    logging.basicConfig(
        filename=log_filename,
        filemode='a',
        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
        datefmt='%H:%M:%S',
        level=logging.DEBUG
    )

    log = logging.getLogger(__name__)
    with open(test_file) as f:
        data = json.load(f)["data"]

    # Set the seeds for reproducibility
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)

    model = LLM(model=model_path, seed=seed)
    sampling_params = SamplingParams(n=1, best_of=1, temperature=temperature, top_p=top_p, stop_token_ids=[2], max_tokens=max_new_tokens)

    if use_fast_kernels:
        """
        Setting 'use_fast_kernels' will enable
        using of Flash Attention or Xformer memory-efficient kernels
        based on the hardware being used. This would speed up inference when used for batched inputs.
        """
        try:
            from optimum.bettertransformer import BetterTransformer

            model = BetterTransformer.transform(model)
        except ImportError:
            log.error(
                "Module 'optimum' not found. Please install 'optimum' it before proceeding."
            )
            results = []
    tokenizer_embedings, model_embedings = get_model_and_tokenizer()
    model_embedings.cuda()

    db = process_data(read_data())
    db_texts = db["information"].values.tolist()[:500]
    db["raw_texts"] = (
        "### Question:"
        + db["question"]
        + "### Choices: "
        + db["choices"]
        + "### Explanation: "
        + db["explanation"]
    )
    db_raw_texts = db["raw_texts"].values.tolist()[:500]
    log.info("Embedding database")
    import gc
    gc.collect()
    db_embeddings = embedding_text(
        tokenizer=tokenizer_embedings, model=model_embedings, input_texts=db_texts
    )
    print(db_embeddings.shape)
    # return None
    import gc
    gc.collect()


    # Dummy
    model.generate(get_user_prompt(data[0]), sampling_params)

    results = []
    times = []

    for idx, example in enumerate(data):
        log.info(f"Processing {idx}")
        start = time.time()
        relevant_examples = get_relevance_embeddings(
            db_embeddings,
            embedding_query_text(
                tokenizer=tokenizer_embedings,
                model=model_embedings,
                query_text=example["question"],
            ),
        )
        log.info("get relevant_question")
        relevant_examples = get_relevance_texts(
            input_texts=db_raw_texts, scores=relevant_examples, top_k=1
        )
        user_prompt = get_user_prompt(
            example, relevant_examples="\n".join(relevant_examples)
        )
        # user_prompt = get_user_prompt(example)
        id = example["id"]
        choices = example["choices"]

        output = model.generate(user_prompt, sampling_params)[0]

        prompt = output.prompt
        gen_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {gen_text!r}")

        answer_text = None

        for text in gen_text.split("###"):
            if 'Final choice' in text:
                answer_text = text
                break

        if answer_text is None:
            answer_text = gen_text

        log.info(f"Output text: {prompt + gen_text}")
        log.info(f"Gen text {gen_text}")
        log.info(f"Answer text {answer_text}")

        answer = post_processing_answer(answer_text, choices)
        e2e_inference_time = int((time.time() - start) * 1000)
        log.info(f'Inference time: {e2e_inference_time}')
        log.info(f"Answer {answer}")
        if answer is None:
            answer = random.choice(choices)
            log.info(f"Random Answer {answer}")
        results.append({"id": id, "answer": answer})
        times.append({"id": id, "time": e2e_inference_time})

    result_df = pd.DataFrame.from_dict(results)
    result_df.to_csv(output_filepath, index=False)

    time_df = pd.DataFrame.from_dict(times)
    time_df.to_csv(time_output_filepath, index=False)

In [ ]:
main(
    model_path='output',
    max_new_tokens=1024,
    temperature=0.1
)

INFO 12-06 14:49:53 llm_engine.py:73] Initializing an LLM engine with config: model='/output', tokenizer='/output', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=42)
INFO 12-06 14:50:07 llm_engine.py:222] # GPU blocks: 2146, # CPU blocks: 2048
(500, 768)


Processed prompts: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.87s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:Bạn Nam mang hai tờ tiền có mệnh giá 10 000 đồng đi mua bút chì. Bạn mua hết 15 000. Bạn Nam còn thừa ...………đồng.### Choices: ['A. 10 000 đồng', 'B. 5 000 đồng', 'C. 2 000 đồng', 'D. 1 000 đồng']### Explanation: Đầu tiên, chúng ta cần hiểu rõ về câu hỏi. Bạn Nam mang theo hai tờ tiền mỗi tờ có mệnh giá 10 000 đồng, tức là tổng cộng bạn Nam có 10 000 đồng x 2 = 20 000 đồng.\n\nSau đó, bạn Nam đã dùng 15 000 đồng để mua bút chì. Vậy, số tiền bạn Nam đã dùng là 15 000 đồng.\n\nCuối cùng, để tìm ra số tiền bạn Nam còn lại sau khi mua bút chì, chúng ta lấy tổng số tiền bạn Nam có ban đầu trừ đi số tiền bạn Nam đã dùng để mua bút chì. Tức là: 20 000 đồng - 15 000 đồng = 5 000 đồng.\n\nVậy, bạn Nam còn thừa 5 000 đồng sau khi mua bút chì. Đáp án chính xác là 5 000 đồng.\n### Questi

Processed prompts: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.78s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:Cả ngày xe đạp A đi được 7676m và xe đạp B đi được 8km. Hỏi quãng đường của xe đạp A hay xe đạp B đi được dài hơn?### Choices: ['A. Xe A', 'B. Xe B', 'C. Cả hai xe đi quãng đường bằng nhau']### Explanation: Để trả lời câu hỏi này, trước hết chúng ta cần chuyển đổi tất cả các đơn vị đo lường về cùng một đơn vị để so sánh. Trong trường hợp này, chúng ta sẽ chuyển đổi tất cả về mét.\n\nXe đạp A đã đi được 7676 mét. Đối với xe đạp B, chúng ta cần chuyển đổi từ km sang mét. Chúng ta biết rằng 1 km = 1000 mét, vì vậy 8 km = 8 * 1000 = 8000 mét.\n\nBây giờ chúng ta có thể so sánh: 7676 mét (xe A) so với 8000 mét (xe B). Rõ ràng, 8000 mét lớn hơn 7676 mét.\n\nVì vậy, xe đạp B đã đi được quãng đường dài hơn.\n### Question: Một người đi xe đạp từ A lúc 7 giờ với vận tốc 12km/h. Đến 8 

Processed prompts: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.10s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:Cạnh của hình lập phương gấp lên 2 lần thì thể tích được gấp lên:### Choices: ['A. 2 lần', 'B. 4 lần', 'C. 6 lần', 'D. 8 lần']### Explanation: Cạnh của hình lập phương gấp lên 2 lần thì thể tích được gấp lên: 2 ${\\times}$ 2 ${\\times}$ 2 = 8 (lần).\n### Question: Cạnh của hình lập phương gấp lên 2 lần thì diện tích xung quanh gấp lên mấy lần?\n### Choices: ['A. 2 lần','B. 4 lần','C. 6 lần','D. 8 lần']\n### Explanation: ", Generated text: 'Để giải thích câu trả lời này, chúng ta cần hiểu rõ về cách tính diện tích xung quanh của hình lập phương.\n\nDiện tích xung quanh của hình lập phương được tính bằng cách cộng diện tích của 4 mặt của hình lập phương lại. Mỗi mặt của hình lập phương là một hình vuông có cạnh bằng cạnh của hình lập phương.\n\nVì vậy, nếu cạn

Processed prompts: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.68s/it]


Prompt: 'Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:“Một trăm hai mươi lăm mét vuông” được viết là:### Choices: [\'A. 125m2\', \'B. 125m\', \'C. 152m\', \'D. 152m2\']### Explanation: Đáp án đúng là 125m2. \n\nGiải thích: \n\nTrước hết, ta cần hiểu rằng "một trăm hai mươi lăm" trong tiếng Việt tương ứng với số 125 trong hệ thống số. \n\nTiếp theo, "mét vuông" là đơn vị đo diện tích. Trong toán học và vật lý, mét vuông (ký hiệu là m2) được sử dụng để đo diện tích của một hình vuông có cạnh dài 1 mét. \n\nVì vậy, khi kết hợp số 125 với đơn vị đo mét vuông, ta được 125m2. \n\nCác lựa chọn khác không đúng vì: \n- 125m là đơn vị đo chiều dài, không phải diện tích.\n- 152m và 152m2 đều không đúng vì số được viết không phải là 125.\n### Question: Một thửa ruộng hình thang có đáy bé dài 8m, đáy lớn dài 12m. Kéo dài đáy lớn thêm 5m thì

Processed prompts:   0%|                                                                       | 0/1 [00:00<?, ?it/s]